In [1]:
import os
import sys
sys.path.append("../Infusion")
from dataset.text2shape_dataset import Text2Shape
from pathlib import Path
from tqdm import tqdm
import torch
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
ds_path = Path("/media/data2/aamaduzzi/datasets/Text2Shape/")

In [3]:
test_ds = Text2Shape(root=ds_path,
                            split="test",
                            categories="all",
                            from_shapenet_v1=True,
                            from_shapenet_v2=False,
                            conditional_setup=True,
                            language_model="t5-11b",
                            lowercase_text=True,
                            max_length=60,
                            padding=False,
                            scale_mode=None)

initializing dataset...
annotations:  /media/data2/aamaduzzi/datasets/Text2Shape/annotations/from_shapenet_v1/test_all.csv


# Text Embeddings Visualization

In [4]:
# get text embeddding
# compute mean
# save to tsv 

log_dir = './'
mean_embeds = []
texts = []
for i in tqdm(range(len(test_ds))):
    text = test_ds[i]["text"]
    text_embed = test_ds[i]["text_embed"]
    sum_embed = torch.sum(text_embed, dim=1)
    # compute mean over non-zero elements
    # get n of nonzero elements
    index = (sum_embed == 0).nonzero()
    if len(index)>0:
        index = index[0]
    else:
        index = text_embed.shape[0]
    
    mean_embed = torch.mean(text_embed[:index], dim=0)
    mean_embeds.append(mean_embed)
    texts.append(text)

print('mean embeds: ', len(mean_embeds))
print('texts: ', len(texts))

100%|██████████| 7453/7453 [00:00<00:00, 8954.84it/s]

mean embeds:  7453
texts:  7453


In [ ]:
# save embeddings in tsv files

with open(os.path.join(log_dir, 'data.tsv'), "w") as f:
  for mean_embed in tqdm(mean_embeds):
    for i in range(mean_embed.shape[0]):
        f.write("{}\t".format(mean_embed[i]))
    f.write("\n")

# save embeddings to tsv file
count=0
with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
  for text in tqdm(texts):
    if "\n" in text:
      count +=1
      text = text.replace('\n', ' ')
      if len(text)<3:
        print('0 len')
    f.write("{}\n".format(text))

In [15]:
# visualization
# the generated tsv files must be loaded to http://projector.tensorflow.org/

# comments: T5EncoderModel produces near embeddings for text prompts with similar syntactic structure, instead of meaning

# Grammar errors Visualization

100%|██████████| 7453/7453 [00:00<00:00, 1431522.08it/s]

63


# Sentence Length Visualization

# Looking for wrong descriptions...